In [2]:
import os
import json
from functools import partial
from collections import defaultdict
from typing import Tuple, List, Optional, Callable
import numpy as np
import torch

from torch.utils import data
from trajdata import UnifiedDataset, SceneBatch, AgentBatch
from trajdata.data_structures.batch_element import SceneBatchElement
from trajdata.data_structures.agent import AgentMetadata, AgentType
from trajdata.data_structures.state import StateArray
from trajdata.augmentation import NoiseHistories
from trajdata.utils.state_utils import transform_state_np_2d

In [3]:
def custom_agent_hist(
    batch_elem: SceneBatchElement,
    history_sec: Tuple[Optional[float], Optional[float]],
) -> np.ndarray:
    """agent’s history transformed into agent-centric coordinates"""
    dt = batch_elem.dt
    max_hist_len: int = round((history_sec[1]/dt)) + 1
    world_agent_hist: List[np.ndarray] = batch_elem.agent_histories
    state_dim = world_agent_hist[0].shape[-1]

    agent_pos_list = [hist[-1,:2] for hist in world_agent_hist]
    agent_sc_list = [hist[-1,-2:] for hist in world_agent_hist]

    agents_hist_st: List[np.ndarray] = []
    for idx, (pos, sc) in enumerate(zip(agent_pos_list, agent_sc_list)):
        cos_agent = sc[-1]
        sin_agent = sc[-2]
        centered_world_from_agent_tf: np.ndarray = np.array(
            [
                [cos_agent, -sin_agent, pos[0]],
                [sin_agent, cos_agent, pos[1]],
                [0.0, 0.0, 1.0],
            ]
        )
        centered_agent_from_world_tf: np.ndarray = np.linalg.inv(
            centered_world_from_agent_tf
        )
        hist = world_agent_hist[idx]
        hist_st = transform_state_np_2d(hist, centered_agent_from_world_tf)

        t_i = hist_st.shape[0]
        if t_i<max_hist_len:
            padding = np.full((max_hist_len-t_i, state_dim), np.nan, dtype=hist_st.dtype)
            hist_st_padded = np.concatenate([padding, hist_st], axis=0)
        else:
            hist_st_padded = hist_st[:max_hist_len]
        agents_hist_st.append(hist_st_padded)

    return np.stack(agents_hist_st, axis=0)

def get_neighs(
    batch_elem: SceneBatchElement,
    interaction_radius: float,
) -> np.ndarray:
    """Provides adjacency matrix"""
    agents: List[AgentMetadata] = batch_elem.agents
    curr_states = []
    for agent in agents:
        raw_state: StateArray = batch_elem.cache.get_raw_state(
            agent.name, batch_elem.scene_ts
        )
        state = np.asarray(raw_state)
        curr_states.append(state)

    is_neigh = []
    for state in curr_states:
        distances = [
            np.linalg.norm(state[:2] - agent_st[:2])
            for agent_st in curr_states
        ]
        is_neigh.append([dist <= interaction_radius for dist in distances])
    is_neigh_mat = np.stack(is_neigh, axis=0)
    np.fill_diagonal(is_neigh_mat, False)

    return is_neigh_mat

def per_agent_neigh_hist(
    batch_elem: SceneBatchElement,
    history_sec: Tuple[Optional[float], Optional[float]],
) -> np.ndarray:
    """
    Provide neighbor history of each agent in scene
    in respective agent-centric frames
    """
    assert batch_elem.standardize_data is False, \
        "Per-agent history requires a non-standarized dataset (set standardize_data=False)"

    dt = batch_elem.dt
    max_hist_len: int = round((history_sec[1]/dt)) + 1
    world_agent_hist: List[np.ndarray] = batch_elem.agent_histories
    state_dim = world_agent_hist[0].shape[-1]
    num_agents = batch_elem.num_agents

    agent_pos_list = [hist[-1,:2] for hist in world_agent_hist]
    agent_sc_list = [hist[-1,-2:] for hist in world_agent_hist]

    neigh_hists: List[List[np.ndarray]] = []
    for idx, (pos, sc) in enumerate(zip(agent_pos_list, agent_sc_list)):
        # calculate transformation matrix to go from world frame to agent frame
        cos_agent = sc[-1]
        sin_agent = sc[-2]
        centered_world_from_agent_tf: np.ndarray = np.array(
            [
                [cos_agent, -sin_agent, pos[0]],
                [sin_agent, cos_agent, pos[1]],
                [0.0, 0.0, 1.0],
            ]
        )
        centered_agent_from_world_tf: np.ndarray = np.linalg.inv(
            centered_world_from_agent_tf
        )

        row_hists: List[Optional[np.ndarray]] = []
        for jdx, agent_hist in enumerate(world_agent_hist):
            # skip self
            if jdx == idx:
                continue
            # append if neighbor
            if batch_elem.extras["is_neigh"][idx, jdx]:
                hist_st = transform_state_np_2d(agent_hist, centered_agent_from_world_tf)
                row_hists.append(hist_st)
            # else append None
            else:
                row_hists.append(None)
        neigh_hists.append(row_hists)

    output = np.full((num_agents, num_agents-1, max_hist_len, state_dim), np.nan, dtype=float)
    # pad arrays to match maximum history length (8 steps),
    # and max possible neighbors (num of agents - 1)
    for i in range(num_agents):
        for k, hist_ij in enumerate(neigh_hists[i]):
            if hist_ij is None:
                continue
            len_j = hist_ij.shape[0]
            output[i, k, -len_j:, :] = hist_ij

    return output

In [4]:
def extras_collate_fn(
    batch_elems: List[SceneBatchElement],
    history_sec: Tuple[Optional[float], Optional[float]],
    base_collate: Callable[[List[SceneBatchElement]], SceneBatch],
) -> SceneBatch:
    """
    1) Pads each extra (“agent_hist_st”, “is_neigh”, “neigh_hist”) 
    2) Calls base_collate(...) (i.e. scene_collate_fn) on the padded batch.
    """
    max_agent_num: int = max(elem.num_agents for elem in batch_elems) # M
    dt = batch_elems[0].dt
    max_hist_len: int = round((history_sec[1]/dt)) + 1
    state_dim = batch_elems[0].agent_histories[0].shape[-1]

    for elem in batch_elems:
        n_i = elem.num_agents
        agent_hist = elem.extras.get("agent_hist_st")
        mat = elem.extras["is_neigh"]
        neigh_hist = elem.extras["neigh_hist"]

        if n_i < max_agent_num:
            # Pad "agent_hist_st": shape (n_i, max_hist_len, 8) -> (M, max_hist_len, 8)
            pad_agent_hist = np.full(
                (max_agent_num-n_i, max_hist_len, state_dim),
                np.nan,
                dtype=agent_hist.dtype
            )
            padded_agent_hist = np.concatenate([agent_hist, pad_agent_hist], axis=0)
            # Pad "is_neigh": shape (n_i, n_i) -> (max_agents_in_batch, max_agents_in_batch)
            pad_mat = np.zeros((max_agent_num, max_agent_num), dtype=mat.dtype)
            pad_mat[:n_i, :n_i] = mat
            elem.extras["is_neigh"] = pad_mat
            # Pad "neigh_hist": shape (n_i, n_i-1, max_hist_len, 8) -> (M, M-1, max_hist_len, 8)
            padded_neigh_hist = np.full(
                (max_agent_num, max_agent_num-1, max_hist_len, state_dim),
                np.nan,
                dtype=neigh_hist.dtype
            )
            padded_neigh_hist[:n_i, :(n_i - 1), :, :] = neigh_hist
        else:
            padded_agent_hist = agent_hist[:max_agent_num]
            elem.extras["is_neigh"] = mat[:max_agent_num, :max_agent_num]
            padded_neigh_hist = neigh_hist[:max_agent_num]

        elem.extras["agent_hist_st"] = padded_agent_hist
        elem.extras["neigh_hist"] = padded_neigh_hist

    return base_collate(batch_elems)

In [5]:
log_dir = '../../../data/trained_models/trajectory_prediction'
model_dir = os.path.join(log_dir, "eth-28_May_2025_10_28_45")

with open(os.path.join(model_dir, 'config.json'), 'r', encoding="utf-8") as config_json:
    hyperparams = json.load(config_json)
# device
hyperparams["device"] = "cpu"
hyperparams["trajdata_cache_dir"] = "../../../data/pedestrian_datasets/.unified_data_cache"

desired_data=[
    "eupeds_eth-train",
]
max_agent_num=20
data_dirs = {
    "eupeds_eth": "../../../data/pedestrian_datasets/eth_ucy_peds",
}

attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
# attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 5.0
interaction_radius = 5.0

history_sec = (0.1, hyperparams["history_sec"])
future_sec = (0.1, hyperparams["prediction_sec"])

input_noise = 0.0
augmentations = list()
if input_noise > 0.0:
    augmentations.append(NoiseHistories(stddev=input_noise))

batch_size = 4

dataset = UnifiedDataset(
    desired_data=desired_data,
    centric="scene",
    # centric="agent",
    max_agent_num=max_agent_num,
    history_sec=history_sec,
    future_sec=future_sec,
    agent_interaction_distances=attention_radius,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    augmentations=augmentations if len(augmentations) > 0 else None,
    standardize_data=False,
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    data_dirs=data_dirs,
    verbose=True,
    extras={
        "agent_hist_st": partial(custom_agent_hist, history_sec=history_sec),
        "is_neigh": partial(get_neighs, interaction_radius=interaction_radius),
        "neigh_hist": partial(per_agent_neigh_hist, history_sec=history_sec),
    }
)

print(f"# Data Samples: {len(dataset):,}")

base_collate = dataset.get_collate_fn(pad_format="right")

dataloader = data.DataLoader(
    dataset,
    # collate_fn=dataset.get_collate_fn(pad_format="right"),
    collate_fn=partial(
            extras_collate_fn,
            history_sec=history_sec,
            base_collate=base_collate
    ),
    pin_memory=False if hyperparams["device"] == "cpu" else True,
    batch_size=batch_size,
    shuffle=True,
    num_workers=hyperparams["preprocess_workers"],
    sampler=None,
)

Loading data for matched scene tags: ['eupeds_eth-zurich-train', 'eupeds_eth-train-cyprus']


Calculating Agent Data (Serially): 100%|██████████| 1/1 [00:00<00:00, 13066.37it/s]

1 scenes in the scene index.



Structuring Scene Data Index: 100%|██████████| 1/1 [00:00<00:00, 28339.89it/s]

# Data Samples: 674


In [6]:
batch: SceneBatch = next(iter(dataloader))

In [7]:
batch.__dict__.keys()

dict_keys(['data_idx', 'scene_ts', 'dt', 'num_agents', 'agent_type', 'centered_agent_state', 'agent_names', 'agent_hist', 'agent_hist_extent', 'agent_hist_len', 'agent_fut', 'agent_fut_extent', 'agent_fut_len', 'robot_fut', 'robot_fut_len', 'map_names', 'maps', 'maps_resolution', 'vector_maps', 'rasters_from_world_tf', 'centered_agent_from_world_tf', 'centered_world_from_agent_tf', 'scene_ids', 'history_pad_dir', 'extras'])

In [8]:
print(f"Num of agents in scenes: {batch.num_agents}")
print(f"Shape of agent_hist_st array: {batch.extras['agent_hist_st'].shape}")
print(f"Shape of is_neigh array: {batch.extras['is_neigh'].shape}")
print(f"Shape of neigh hist array: {batch.extras['neigh_hist'].shape}")

Num of agents in scenes: tensor([2, 7, 4, 1])
Shape of agent_hist_st array: torch.Size([4, 7, 8, 8])
Shape of is_neigh array: torch.Size([4, 7, 7])
Shape of neigh hist array: torch.Size([4, 7, 6, 8, 8])


In [31]:
class SceneAgentBatch:
    def __init__(self, batch: SceneBatch):
        self.batch = batch
        self.num_agents = self.batch.num_agents
        self.num_scenes = self.num_agents.shape[0]
        self.agent_hist_st = batch.extras['agent_hist_st']
    
    @property
    def dt(self) -> torch.Tensor:
        return self.batch.dt

    @property
    def agent_hist(self) -> torch.Tensor:
        agent_hist: List[torch.Tensor] = []
        for idx, num_agents in enumerate(self.batch.num_agents):
            for jdx in range(num_agents):
                agent_hist.append(self.agent_hist_st[idx, jdx])

        return torch.stack(agent_hist, dim=0)
    
    @property
    def agent_hist_len(self) -> torch.Tensor:
        return batch.agent_hist_len[batch.agent_hist_len!=0]
    
    @property
    def num_neigh(self) -> torch.Tensor:
        is_neigh = self.batch.extras['is_neigh']
        num_neigh: List[torch.Tensor] = []
        for idx in range(self.num_scenes):
            n_i = self.num_agents[idx].item()
            submat = is_neigh[idx, :n_i, :n_i]
            counts_per_agent = submat.sum(dim=1).to(torch.long)
            num_neigh.append(counts_per_agent)
        return torch.cat(num_neigh, dim=0)
    
    @property
    def neigh_hist(self) -> torch.Tensor:
        grouped_neigh_hist = self.batch.extras['neigh_hist']
        max_num_neigh: int = self.num_neigh.max().item()

        neigh_hist: List[torch.Tensor] = []
        for scene_idx in range(self.num_scenes):
            n_i = self.num_agents[scene_idx].item()
            for agent_idx in range(n_i):
                neigh_hist.append(grouped_neigh_hist[scene_idx, agent_idx, :max_num_neigh])
        
        return torch.stack(neigh_hist)

    @property
    def neigh_hist_len(self) -> torch.Tensor:
        is_neigh = self.batch.extras['is_neigh']
        batch_size = self.agent_hist.shape[0]
        max_num_neigh: int = self.num_neigh.max().item()

        device = self.agent_hist_len.device
        dtype = self.agent_hist_len.dtype
        
        neigh_hist_len_list: List[torch.Tensor] = []
        for scene_idx in range(self.num_scenes):
            n_i = self.num_agents[scene_idx].item()
            for agent_idx in range(n_i):
                neigh_mask = is_neigh[scene_idx, agent_idx, :n_i]
                neighbor_idxs = torch.nonzero(neigh_mask, as_tuple=False).view(-1)
                hist_lens = self.batch.agent_hist_len[scene_idx, neighbor_idxs]
                neigh_hist_len_list.append(hist_lens)

        output = torch.zeros((batch_size, max_num_neigh), dtype=dtype, device=device)

        for idx, hist_lens in enumerate(neigh_hist_len_list):
            k = hist_lens.size(0)
            if k > 0:
                output[idx, :k] = hist_lens.to(device)

        return output
    
    @property
    def neigh_types(self) -> torch.Tensor:
        is_neigh = self.batch.extras['is_neigh']
        agent_types = self.batch.agent_type
        batch_size = self.agent_hist.shape[0]
        max_num_neigh: int = self.num_neigh.max().item()

        device = self.agent_hist_len.device
        dtype = self.agent_hist_len.dtype

        neigh_types_list: List[torch.Tensor] = []
        for scene_idx in range(self.num_scenes):
            n_i = self.num_agents[scene_idx].item()
            for agent_idx in range(n_i):
                neigh_mask = is_neigh[scene_idx, agent_idx, :n_i]
                neighbor_idxs = torch.nonzero(neigh_mask, as_tuple=False).view(-1)
                row_types = agent_types[scene_idx, neighbor_idxs]
                neigh_types_list.append(row_types)

        output = torch.full(
            (batch_size, max_num_neigh),
            -1.0,
            dtype=dtype,
            device=device
        )

        for idx, types in enumerate(neigh_types_list):
            k = types.size(0)
            if k > 0:
                output[idx, :k] = types.to(device)

        return output

In [17]:
scene_batch = SceneAgentBatch(batch)

In [32]:
## Helper Functions
# To Calculate Number of Samples

import warnings

def bisection(N_low, N_high, sample_function):
    """
    :param N_low: Low guess for the intersection
    :param N_high: High guess for the intersection
    :param sample_function: Function for which to find the zero crossing
    :return: The zero crossing
    """
    # print(f"Running bisection to find S between {N_low} and {N_high}...", end="")

    a = N_low
    b = N_high
    f = sample_function

    TOL = 1e-9

    if a > b:
      print('Error: a > b!')
      return -1

    value_a = f(a)

    for i in range(1000):
      c = (a + b) / 2.
      value_c = f(c)

      if value_c == 0 or (b - a) / 2. < TOL:
          # print(f" Found {c:.2f}")
          return c

      if np.sign(value_c) == np.sign(value_a):
          a = c
          value_a = value_c
      else:
          b = c

    print("Bisection failed to converge!")

def nchoosek_rooted(n, k, root):
    """
    To avoid numerical errors, an implementation of N choose K where the root is resolved in between
    :return: (n choose k)^(1/root)
    """
    y = 1.0
    for i in range(k):
        y = y * ((n - (k - i - 1.)) / (k - i))**(1. / root)
    return y

def compute_risk(S, confidence, support):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    return 1. - ((confidence / S) ** (1. / (S - support))) * (1. / nchoosek_rooted(S, support, S - support))

def find_sample_size(support, risk, confidence):
  risk_function = lambda S, support: compute_risk(S, confidence, support)      # Scenario Optimization guarantee for any support
  max_risk_function = lambda S: risk - risk_function(S, support)               # "" for the specified support limit

  S_low = support+1
  S_high = 500000

  S_double = bisection(S_low, S_high, max_risk_function)
  S = np.floor(S_double) + 1

  return S

In [9]:
## Helper functions
# To ensure humans do not collide with the robot during initialization
# and are within field of view of the robot

def is_within_distance(pos1, pos2, distance):
    return np.linalg.norm(np.array(pos1) - np.array(pos2)) <= distance

def angle_between(v1, v2):
    """Calculate the angle between two vectors."""
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    cross = np.cross(np.append(v1_u, 0), np.append(v2_u, 0))
    if cross[2] < 0:
        angle = -angle
    return angle

def is_within_fov(robot_pos, robot_orientation, target_pos, fov=90):
    """Check if target_pos is within the field of view of the robot."""
    direction_vector = np.array([np.cos(np.deg2rad(robot_orientation)), np.sin(np.deg2rad(robot_orientation))])
    target_vector = np.array(target_pos) - np.array(robot_pos)
    angle = np.rad2deg(angle_between(direction_vector, target_vector))
    return -fov/2 < angle < fov/2

# To generate constraint coefficients given human and robot positions
def get_constraint_coeff(robot_pos, human_pos, r):
    u = human_pos - robot_pos
    A = u / np.linalg.norm(u)
    a = A[0]
    b = A[1]
    c = r - A[0]*human_pos[0] - A[1]*human_pos[1]
    return np.array([a, b, c]) # Linear inequality: ax + by + c <= 0

In [10]:
def check_symmetric(matrix):
    """
    Checks if the given matrix is symmetric (A == A^T).
    Raises a ValueError if it is not symmetric.
    """
    if not np.allclose(matrix, matrix.T):  # Checks if A == A^T (within numerical tolerance)
        raise ValueError("Matrix is not symmetric!")

In [ ]:
from pathlib import Path

from traj_pred.modules import ModelRegistrar
from traj_pred import TrajectoryPredictor

# Function to load the trajectron model
def load_model(epoch: int):
    while epoch > 0:
        save_path = Path(model_dir) / f'model_registrar-{epoch}.pt'
        if save_path.is_file():
            break
        epoch -= 1

    model_registrar = ModelRegistrar(model_dir, hyperparams["device"])

    model = TrajectoryPredictor(
        model_registrar=model_registrar,
        hyperparams=hyperparams,
        log_writer=None,
        device=hyperparams["device"])
    model.set_environment()

    checkpoint = torch.load(save_path, map_location=hyperparams["device"])
    model.load_state_dict(checkpoint["model_state_dict"], strict=False)

    return model

epoch = 15

traj_predictor = load_model(epoch)

No traceback available to show.


In [ ]:
## Helper Function
## To get predictions for a scene
## and calculate the constraint coefficients

col_radius = 1.0  # Collision radius for agents
buffer_dist = 0.5 # Buufer distance considered when choosing robot position
vision_radius = 12.0  # Vision radius for robots
ph = 12

def get_scene_graph_data(batch: SceneBatch, scene_data):
    enc = traj_predictor.get_encoding(batch) # TODO
    preds = traj_predictor.incremental_forward(
        batch=batch,
        prediction_horizon=ph,
        num_samples=1,
        full_dist=True
    )

    # All humans in the scene
    scene_data["Human Positions"] = obs.curr_agent_state[:, :2].cpu().numpy()

    # Agent to World frame transformations
    agents_to_world_tf = obs.agents_from_world_tf[:, :2, :2].cpu().numpy()
    # Append predicted positions of humans to occupied positions
    occupied_pos = [pos for pos in scene_data["Human Positions"]]
    # Initiate adjaceny matrix
    adjacency_matrix = np.zeros((len(obs.agent_name), len(obs.agent_name)), dtype=int)

    # Loop through all humans
    for idx, human in enumerate(obs.agent_name):
        # Append predicted positions to occupied positions (after transformation to world frame)
        pred_world_frame = pred[human].reshape(prediction_horizon, 2) @ agents_to_world_tf[idx, :, :] + scene_data["Human Positions"][idx, :]
        for jdx in range(prediction_horizon):
            occupied_pos.append(pred_world_frame[jdx, :])
        # Create adjacency matrix
        current_state = scene_data["Human Positions"][idx, :2]
        agent_to_world_tf = agents_to_world_tf[idx, :, :]
        if obs.num_neigh[idx] > 0:
            for neigh_pos in obs.neigh_hist[idx, :obs.num_neigh[idx].item(), -1, :2].cpu().numpy():
                neigh_pos = neigh_pos @ agent_to_world_tf + current_state
                for jdx, agent_pos in enumerate(obs.curr_agent_state[:, :2].cpu().numpy()):
                    if np.linalg.norm(neigh_pos - agent_pos) < 1e-5:
                        adjacency_matrix[idx, jdx] = 1
        # Store human encodings
        scene_data["Encodings"].append(enc[human].cpu().numpy().flatten())

    scene_data["Adjacency Matrix"] = adjacency_matrix
    check_symmetric(scene_data["Adjacency Matrix"])

    # Assign appropriate robot position
    # print("Occupied positions:", occupied_pos)
    while True: # Generate a random position for the robot
        # Ensure the robot's position does not overlap with any occupied positions
        robot_position = (random.uniform(-vision_radius, vision_radius), random.uniform(-vision_radius, vision_radius))
        if not any(is_within_distance(robot_position, pos, col_radius+buffer_dist) for pos in occupied_pos):
            break
    scene_data["Robot Pose"] = [robot_position[0], robot_position[1]]

    return obs, scene_data


def get_constr_data(t, S, obs: AgentBatch, scene_data):

    _, preds, _, _ = trajectron.incremental_forward(
        timestep=t,
        obs=obs,
        maps=None,
        prediction_horizon=prediction_horizon,
        num_samples=int(S),
        full_dist=False
    )

    # Agent to World frame transformations
    agents_to_world_tf = obs.agents_from_world_tf[:, :2, :2].cpu().numpy()

    constraint_coeffs = []
    for idx, human in enumerate(obs.agent_name):
        # print("Pred shape:", preds[human].shape)
        constraint_pos_st = preds[human][:, :, 0, :].reshape(int(S), 2) # TODO: change the timestpe of prediction positions being considered
        constraint_pos = constraint_pos_st @ agents_to_world_tf[idx, :, :] + scene_data["Human Positions"][idx, :]
        # print("Constraint positions:", constraint_pos)
        for sidx in range(int(S)):
            constraint_coeffs.append(get_constraint_coeff(scene_data["Robot Pose"], constraint_pos[sidx, :], col_radius))
            # print("Constraint Coeffs:", constraint_coeffs[-1])
    scene_data["Constraint Coefficients"] = constraint_coeffs
  
    return scene_data

In [ ]:
# Test constraint calculation
scene_data: Dict[str, List] = {
                "Robot Pose": None,
                "Human Positions": [],
                "Adjacency Matrix": None,
                "Encodings": [],
                }
test_t=287
obs, scene_data = get_scene_graph_data(test_t, scene_data)

In [ ]:
scene_data["Constraint Coefficients"] = None
scene_data = get_constr_data(test_t, 10, obs, scene_data)